In [7]:
# Imports
import os
import pandas as pd
from datetime import datetime
import numpy as np

In [10]:
#Constants and Vars
TC_CSX = {'hp': 158, 'torque': 139, 'mass':1313}
TC = TC_CSX
hpmratio = TC['hp'] / TC['mass']
tmratio = TC['torque'] / TC['mass']


DATA_FILE = '26'
DATA_FILES = [26,27,28,31,32,33,34,36,37,38,40]
DATA_FOLDER = 'hwd_data2'
FILE_NAME = "{}/run_{}.csv".format(DATA_FOLDER, DATA_FILE)
SAVE_FOLDER = 'hwd_processed'
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)

In [11]:
# df_in = pd.read_csv(FILE_NAME,header=None,sep=',', names=['date', 'speed', 'engine_load','throttle','rpm',
#                                                       'heading', 'roll', 'pitch',
#                                                       'sys', 'gyro', 'accel', 'mag',
#                                                       'magx', 'magy', 'magz',
#                                                       'gyrx', 'gyry', 'gyrz',
#                                                       'linaccx', 'linaccy', 'linaxxz'
#                                                      ])

In [12]:
def format_df(f_name):
    df = pd.read_csv(FILE_NAME,header=None,sep=',', names=['date', 'speed', 'engine_load','throttle','rpm',
                                                      'heading', 'roll', 'pitch',
                                                      'sys', 'gyro', 'accel', 'mag',
                                                      'magx', 'magy', 'magz',
                                                      'gyrx', 'gyry', 'gyrz',
                                                      'linaccx', 'linaccy', 'linaxxz'
                                                     ])
    df['time'] = (df.date - df.date.iloc[0])
    df['time_diff'] = df.time.diff()
    df['speed'] = df.speed.str.replace(' kph','')
    df['engine_load'] = df.engine_load.str.replace(' percent', '')
    df['throttle'] = df.throttle.str.replace(' percent', '')
    df['rpm'] = df.rpm.str.replace(' revolutions_per_minute', '')

    obd2_cols = ['speed', 'engine_load', 'throttle', 'rpm']
    for col in obd2_cols:
        df[col].replace('None', np.nan,inplace=True)
        df[col] = df[col].apply(np.float64)
        df[col] = df[col].interpolate(limit=4,direction='both')
    df = df[1:-1]

    min_throttle = df.throttle.min()
    df['throttle'] = df['throttle'] * (1+0.01*min_throttle) - min_throttle
    df = df[df.speed > 0]

    df['lateral_velocity'] = df.heading.diff() / df.time_diff
    df['speed_diff'] = df.speed.diff()
    return df

In [13]:
# HP
test_df = format_df(FILE_NAME)
test_hp = test_df[test_df.rpm > 2500][test_df.speed_diff >= 0]
test_hp['hp_score'] = test_hp.speed_diff * test_hp.throttle
print('HP_SCORE: ',test_hp['hp_score'].mean()/hpmratio)
# Breaking on speed diff??? Then calculate acceleration based on derivative?

HP_SCORE:  83.2563479442


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
test_hp

,date,speed,engine_load,throttle,rpm,heading,roll,pitch,sys,gyro,...,gyry,gyrz,linaccx,linaccy,linaxxz,time,time_diff,lateral_velocity,speed_diff,hp_score
305,1.520612e+09,35.000000,66.666667,14.971165,2510.000000,20.4375,-24.0000,-26.5000,2.0,0.0,...,-0.005556,-0.112222,-0.80,0.35,0.31,47.95,0.14,3.125003,0.000000,0.000000
306,1.520612e+09,36.000000,55.882353,13.628604,2517.000000,20.7500,-24.3125,-26.3750,2.0,0.0,...,-0.016667,-0.100000,-0.49,0.32,0.11,48.09,0.14,2.232141,1.000000,13.628604
388,1.520612e+09,21.000000,58.431373,14.523645,2586.000000,5.5625,-35.5000,-18.5000,0.0,0.0,...,0.186667,0.357778,0.62,-2.37,1.01,59.64,0.15,-20.833353,0.000000,0.000000
389,1.520612e+09,22.000000,58.431373,14.076125,2643.000000,2.0625,-35.6875,-18.9375,0.0,0.0,...,0.090000,0.327778,0.25,-2.37,1.17,59.79,0.15,-23.333318,1.000000,14.076125
390,1.520612e+09,23.000000,58.431373,14.523645,2723.000000,358.4375,-35.4375,-19.1875,0.0,0.0,...,0.084444,0.422222,0.39,-2.43,1.93,59.96,0.17,2096.322589,1.000000,14.523645
391,1.520612e+09,24.000000,57.254902,14.523645,2817.000000,354.5000,-34.8125,-18.4375,0.0,0.0,...,0.113333,0.325556,1.29,-2.93,0.67,60.11,0.15,-26.250025,1.000000,14.523645
392,1.520612e+09,24.000000,56.078431,14.523645,2848.000000,350.8750,-35.0000,-18.1250,0.0,0.0,...,0.128889,0.313333,1.30,-3.87,1.36,60.25,0.14,-25.892838,0.000000,0.000000
393,1.520612e+09,25.000000,56.078431,14.747405,2873.000000,347.5000,-35.0625,-18.7500,0.0,0.0,...,0.110000,0.455556,1.03,-3.53,3.31,60.38,0.13,-25.961563,1.000000,14.747405
394,1.520612e+09,26.000000,61.568627,14.971165,2670.000000,343.7500,-34.8750,-19.3750,0.0,0.0,...,0.122222,0.406667,0.87,-2.90,1.46,60.50,0.12,-31.249968,1.000000,14.971165
406,1.520612e+09,34.000000,74.901961,18.551326,2506.000000,309.2500,-33.2500,-21.0625,0.0,0.0,...,0.064444,0.214444,-0.01,-1.92,0.01,62.06,0.13,-11.538451,0.000000,0.000000


In [16]:
for f in DATA_FILES:
    print('RUN: {}'.format(f))
    FILE_LOC = "{}/run_{}.csv".format(DATA_FOLDER, f)
    df = format_df(FILE_LOC)
    df_hp = df[df.rpm > 2500][df.speed_diff >= 0]
    df_hp['hp_score'] = df_hp.speed_diff * df_hp.throttle
    print('HP_SCORE: ',df_hp['hp_score'].mean()/hpmratio)
    
    df_torque = df[df.rpm < 3000][df.speed_diff >= 0]
    df_torque['torque_score'] = df_torque.speed_diff * df_torque.throttle
    print('TORQUE_SCORE: ',df_torque['torque_score'].mean()/tmratio)
    
    print('\n')
    

RUN: 26
HP_SCORE:  83.2563479442
TORQUE_SCORE:  26.7183964073


RUN: 27


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


HP_SCORE:  83.2563479442
TORQUE_SCORE:  26.7183964073


RUN: 28
HP_SCORE:  83.2563479442
TORQUE_SCORE:  26.7183964073


RUN: 31
HP_SCORE:  83.2563479442
TORQUE_SCORE:  26.7183964073


RUN: 32
HP_SCORE:  83.2563479442
TORQUE_SCORE:  26.7183964073


RUN: 33
HP_SCORE:  83.2563479442
TORQUE_SCORE:  26.7183964073


RUN: 34
HP_SCORE:  83.2563479442
TORQUE_SCORE:  26.7183964073


RUN: 36
HP_SCORE:  83.2563479442
TORQUE_SCORE:  26.7183964073


RUN: 37
HP_SCORE:  83.2563479442
TORQUE_SCORE:  26.7183964073


RUN: 38
HP_SCORE:  83.2563479442
TORQUE_SCORE:  26.7183964073


RUN: 40
HP_SCORE:  83.2563479442
TORQUE_SCORE:  26.7183964073




In [160]:
df[df.speed > 120]

,date,speed,engine_load,throttle,rpm,heading,roll,pitch,sys,gyro,...,gyrx,gyry,gyrz,linaccx,linaccy,linaxxz,time,time_diff,lateral_velocity,speed_diff
3778,1.520613e+09,121.000000,97.254902,75.386390,5559.0,46.8125,-34.1875,-17.5625,1.0,3.0,...,0.097778,0.007778,0.006667,0.25,-1.09,0.18,535.06,0.16,0.000000,1.000000
3779,1.520613e+09,122.000000,96.078431,75.386390,5601.0,46.8125,-34.1250,-17.4375,1.0,3.0,...,-0.026667,0.000000,-0.014444,0.11,0.05,-0.12,535.23,0.17,0.000000,1.000000
3780,1.520613e+09,122.333333,96.470588,75.386390,5622.0,46.7500,-34.1250,-17.2500,1.0,3.0,...,0.021111,-0.013333,0.006667,0.34,-0.19,-0.17,535.37,0.14,-0.446429,0.333333
3781,1.520613e+09,122.666667,96.470588,75.386390,5647.5,46.6250,-34.0625,-17.1875,1.0,3.0,...,-0.062222,-0.010000,0.003333,0.44,-0.53,0.17,535.54,0.17,-0.735294,0.333333
3782,1.520613e+09,123.000000,98.431373,75.386390,5673.0,46.5000,-34.0625,-17.1250,2.0,3.0,...,-0.010000,0.007778,0.004444,-0.39,-0.19,0.94,535.68,0.14,-0.892858,0.333333
3783,1.520613e+09,124.000000,96.862745,75.386390,5716.0,46.4375,-34.1875,-17.0000,2.0,3.0,...,0.052222,0.015556,0.003333,-0.28,-1.14,0.16,535.83,0.15,-0.416666,1.000000
3784,1.520613e+09,125.000000,99.215686,75.386390,5725.0,46.3750,-34.2500,-17.0625,2.0,3.0,...,-0.025556,-0.005556,0.014444,0.07,-1.02,0.56,535.97,0.14,-0.446428,1.000000
3785,1.520613e+09,125.000000,99.215686,75.386390,5747.0,46.1875,-34.1875,-17.0625,2.0,3.0,...,-0.085556,-0.024444,0.024444,-0.36,-1.68,0.31,536.14,0.17,-1.102942,0.000000
3786,1.520613e+09,126.000000,96.862745,75.386390,5782.0,45.9375,-34.2500,-16.8750,2.0,3.0,...,-0.092222,-0.023333,0.030000,0.36,-1.39,-0.33,536.30,0.16,-1.562499,1.000000
3787,1.520613e+09,127.000000,96.470588,75.386390,5797.5,45.6250,-34.1875,-16.8125,2.0,3.0,...,-0.016667,-0.014444,0.028889,0.32,-0.91,-0.48,536.42,0.12,-2.604169,1.000000
